# Project 3 - Book 2: Web APIs & Classification

## Problem Statement

Nutrino is a leading provider of nutrition related data services and analytics. As part of the data science team, we have been tasked to generate business insights curated from popular social media platforms. The company will be able to use that information to better understand customers & markets, enhance decision-making, and ultimately increase profitability.

To do so, we will first be scrapping data from reddit and using classification models such as Logistic Regression and Naive Bayes to uncover patterns within 2 popular diets, Keto and Vegan. In developing this proof of concept, we want to be able to classify all available text data into business ready data, stengthening our core analytics product. 

We also hope to reveal previously unrecognised sub-trends that pertains to attitudes, lifestyles and buying behavior. With a better understanding of the population and their eating patterns, we will be able to advise our clients oh how they can launch their targeted marketing campaigns and improve the success of their products and programs.


## Executive Summary

As the data science team in Nutrino, we have been tasked to build a classifier to improve core product of the company, which is to provide nutrition related data services and analytics. We are also tasked to identify patterns on 2 currently trending diets, keto and vegan. 

Our classifier was successful in predicting at an above 90% accuracy score. We also identified patterns in the motivations and preferences of the 2 groups of subredditors, which will help determine the kind of customer engagement with teach group. 



## Notebooks:
- [Data Scrapping and Cleaning](./book1_data_scrapping_cleaning.ipynb)
- [EDA](./book2_eda.ipynb)
- [Modeling and Recommendations](./book3_preprocesing_modeling_recommendations.ipynb)


## Contents:
- [Import Libraries](#Import-Libraries)
- [Import Data](#Import-Data)
- [Baseline Accuracy](#Baseline-Accuracy)
- [Model Prep](#Model-Prep)
- [Model Selection](#Model-Selection)
- [Modeling Test Data](#Modeling-Test-Data)
- [Classification Metrics](#Classification-Metrics)
- [Recommendations](#Recommendations)
- [Conclusion](#Conclusion)

### Import Libraries

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

### Import data

In [2]:
df = pd.read_csv('../datasets/data_clean.csv')

In [3]:
df.shape

(1644, 8)

In [4]:
df.head()

,text,vegan_label,subred_name,author,upvotes,num_comments,post_id,word_count
0,vegan hacktivists looking developers ui design...,1,r/vegan,veganactivismbot,76,0,f3svif,184
1,last words fellow vegan elijah mcclain murdere...,1,r/vegan,VenmoMeFiveBucks,5114,409,hf6eej,13
2,promising future think,1,r/vegan,The_Shorey,2709,151,hfkmlc,3
3,vegan mean,1,r/vegan,benwaboekar,188,40,hfl3xp,2
4,suddenly finding smell cooking meat repulsive ...,1,r/vegan,sneakyt123,38,11,hfn5ku,38


In [5]:
df.isnull().sum()
#no null values

text            0
vegan_label     0
subred_name     0
author          0
upvotes         0
num_comments    0
post_id         0
word_count      0
dtype: int64

### Baseline Accuracy

In [6]:
#let's calculate a baseline score so that we know if 
#our model is outperforming our null model

df['vegan_label'].value_counts(normalize=True)

1    0.587591
0    0.412409
Name: vegan_label, dtype: float64

#### Interpretation of baseline accuracy
The baseline accuracy means that if we predict 1 for all posts, we would be right at least 57% of the time. 

Now let's prepare for modelling!

### Model Prep

#### Model Prep: Create feature matrix (`X`) and target vector (`y`)

In [7]:
X=df['text']
y=df['vegan_label']

#### Model Prep: Train/test split

In [8]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,stratify=y)
#here we stratify due to the unbalanced classes in the dataset

#### Model Prep: Steps for Pipeline

In [10]:
transformer = [('cvec', CountVectorizer()),('tvec',TfidfVectorizer())]
model = [('lr', LogisticRegression()),('nb', MultinomialNB())]

In [12]:
trans_pipe_params = {
    'cvec__max_features': [1000, 1500, 2000],
    'cvec__min_df': [2, 3],
    'cvec__max_df': [0.8, 0.85, 0.9],
    'cvec__ngram_range': [(1,1), (1,2)],
    'tvec__max_features': [2500, 3000, 3500],
    'tvec__max_df': [0.3, 0.5, 0.7],
    'tvec__sublinear_tf': [True, False],
    'tvec__ngram_range': [(1,1), (1,2)]
}

In [21]:
trans_pipe = Pipeline(transformer)
trans_pipe.fit_transform(X_train)

AttributeError: lower not found

In [15]:
model_pipe_params = {
    'lr__penalty': ['l1', 'l2'],
    'lr__solver' : ['liblinear'],
    'lr__C':[1,0.1,0.01], 
    'nb__alpha' : [1.0, 1.1, 1.2],
    'nb__fit_prior' : [True, False]
}

In [ ]:
model_pipe = Pipeline(model)
opt

In [9]:
#here we prepare the steps for pipeline
# cvec_lr = [('cvec', CountVectorizer()),('lr', LogisticRegression())]
# tvec_lr = [('tvec',TfidfVectorizer()),('lr', LogisticRegression())]
# cvec_nb = [('cvec', CountVectorizer()),('nb', MultinomialNB())]
# tvec_nb = [('tvec',TfidfVectorizer()),('nb', MultinomialNB())]

### Model Selection

#### Choice of Model

In order to select a model, we will be running 2 vectorizers and 2 models. 
The vectorizers are namely Count Vectorizer and TFIDF Vectorizer.
Count Vectoriser counts the number of times a word appears. 
TFIDF goes a step further to apply a penalty for words that appear in  multiple posts. 

The models we chose are Logistic Regression and Naive Bayes (MultiNomial). 

Logistic Regression estimates the relationship between our features and the target variable by estimating probabilities using a sigmoid function.
Naive bayes classifier is based on the bayes theorem which is in turn baed on conditional probabilities. It predicts probailities for each class and the class with the highest probability is ocnsidered the most likely class. 

We chose log reg because:
- Y is binary
- easy to interpret

We chose NB because:
- performs well with text classification
- specifically the Multinomial because the columns of X are all integer counts


### Model 1: CountVectorizer and Logistic Regression

#### Model Prep: Pipe_params

In [10]:
pipe_1_params = {
    'cvec__max_features': [1000, 1500, 2000],
    'cvec__min_df': [2, 3],
    'cvec__max_df': [0.8, 0.85, 0.9],
    'cvec__ngram_range': [(1,1), (1,2)],
    'lr__penalty': ['l1', 'l2'],
    'lr__solver' : ['liblinear'],
    'lr__C':[1,0.1,0.01]
}

#### Model Fit and Score: Train Data

In [11]:
%%time

#instantiate pipeline
pipe_1 = Pipeline(cvec_lr)

gs_1 = GridSearchCV(pipe_1,param_grid=pipe_1_params,cv=5)
gs_1.fit(X_train,y_train)

print(gs_1.best_score_)
print(gs_1.best_params_)

0.9596958174904943
{'cvec__max_df': 0.8, 'cvec__max_features': 1000, 'cvec__min_df': 2, 'cvec__ngram_range': (1, 2), 'lr__C': 0.01, 'lr__penalty': 'l2', 'lr__solver': 'liblinear'}
CPU times: user 3min 52s, sys: 1.96 s, total: 3min 54s
Wall time: 3min 57s


### Model 2: Tfidf Vectorizer and Logistic Regression

#### Model Prep: Pipe_params

In [12]:
pipe_2_params = {
    'tvec__max_features': [2500, 3000, 3500],
    'tvec__max_df': [0.3, 0.5, 0.7],
    'tvec__sublinear_tf': [True, False],
    'tvec__ngram_range': [(1,1), (1,2)],
    'lr__penalty': ['l1', 'l2'],
    'lr__solver' : ['liblinear'],
    'lr__C':[1,0.1,0.01]
}

#### Model Fit and Score: Train Data

In [13]:
%%time
#instantiate pipeline
pipe_2 = Pipeline(tvec_lr)

gs_2 = GridSearchCV(pipe_2,param_grid=pipe_2_params,cv=5)
gs_2.fit(X_train,y_train)

print(gs_2.best_score_)
print(gs_2.best_params_)

0.9589353612167301
{'lr__C': 1, 'lr__penalty': 'l2', 'lr__solver': 'liblinear', 'tvec__max_df': 0.5, 'tvec__max_features': 3500, 'tvec__ngram_range': (1, 2), 'tvec__sublinear_tf': True}
CPU times: user 4min 24s, sys: 4.24 s, total: 4min 28s
Wall time: 4min 33s


### Model 3: CountVectorizer and Multinomial Naive Bayes

#### Model Prep: Pipe_params

In [14]:
pipe_3_params = {
    'cvec__max_features': [1000, 1500, 2000],
    'cvec__min_df': [2, 3],
    'cvec__max_df': [0.75, 0.8, 0.85],
    'cvec__ngram_range': [(1,1), (1,2)],
    'nb__alpha' : [1.0, 1.1, 1.2],
    'nb__fit_prior' : [True, False]
}


#### Model Fit and Score: Train Data

In [15]:
%%time

#instantiate pipeline
pipe_3 = Pipeline(cvec_nb)

gs_3 = GridSearchCV(pipe_3,param_grid=pipe_3_params,cv=5)
gs_3.fit(X_train,y_train)

print(gs_3.best_score_)
print(gs_3.best_params_)

0.9505703422053232
{'cvec__max_df': 0.75, 'cvec__max_features': 1000, 'cvec__min_df': 3, 'cvec__ngram_range': (1, 2), 'nb__alpha': 1.0, 'nb__fit_prior': True}
CPU times: user 4min 22s, sys: 4.48 s, total: 4min 27s
Wall time: 4min 32s


### Model 4: Tfidf Vectorizer and Multinomial Naive Bayes

#### Model Prep: Pipe_params

In [16]:
pipe_4_params = {
    'tvec__max_features': [2500, 3000, 3500],
    'tvec__max_df': [0.3, 0.5, 0.7],
    'tvec__sublinear_tf': [True, False],
    'tvec__ngram_range': [(1,1), (1,2)],
    'nb__alpha' : [1.0, 1.1, 1.2],
    'nb__fit_prior' : [True, False]
}

#### Model Fit and Score: Train Data

In [17]:
%%time
#instantiate pipeline
pipe_4 = Pipeline(tvec_nb)

gs_4 = GridSearchCV(pipe_4,param_grid=pipe_4_params,cv=5)
gs_4.fit(X_train,y_train)

print(gs_4.best_score_)
print(gs_4.best_params_)

0.9437262357414449
{'nb__alpha': 1.0, 'nb__fit_prior': True, 'tvec__max_df': 0.5, 'tvec__max_features': 3500, 'tvec__ngram_range': (1, 2), 'tvec__sublinear_tf': False}
CPU times: user 4min, sys: 3.21 s, total: 4min 3s
Wall time: 4min 5s


#### Scores recap:

In [18]:
print(f'CVec and LR Score: {gs_1.best_score_}')
print(f'TFIDFVec and LR Score: {gs_2.best_score_}')
print(f'CVec and NB Score: {gs_3.best_score_}')
print(f'TFIDFVec and NB Score: {gs_4.best_score_}')

CVec and LR Score: 0.9596958174904943
TFIDFVec and LR Score: 0.9589353612167301
CVec and NB Score: 0.9505703422053232
TFIDFVec and NB Score: 0.9437262357414449


##### Observation on train scores:
Overall, all our scores are pretty similar and way above our baseline score of 0.571. The default score used by gridsearch is the accuracy score. Our accuracy scores range from 0.945 to 0.959. This means that our model is doing a pretty great job of classifying text from the 2 subreddits. 

##### Model Selection:
Based on the cross validation scores from all 4 models, we will proceed with the Count Vectorizer and Logistic Regression combination, which gave us the highest train accuracy score of 0.959. 

### Modelling Test Data

In [19]:
gs_1.score(X_test,y_test)

0.9604863221884499

In [33]:
opti_cvec = CountVectorizer(max_df =0.8, max_features=1000, min_df=2, ngram_range=(1, 2))
X_test_cvec = opti_cvec.fit_transform(X_test).todense()

In [35]:
lr = LogisticRegression(C=0.01, penalty='l2', solver='liblinear')
lr.fit(X_train_cvec,y_train)
lr.score(X_test_cvec,y_test)

0.9787234042553191

##### Observation of test scores
Our test score exceeded our train score with 0.964, with some variance from the train score. 


#### Interpretation of coefficients

In [20]:
wd_features = gs_1.best_estimator_.named_steps['cvec'].get_feature_names()
wd_coef = gs_1.best_estimator_.named_steps['lr'].coef_[0]

coef_df = pd.DataFrame(zip(wd_features,wd_coef),columns=['features','coef']).sort_values('coef',ascending=False)

print(coef_df.head(2))
print(coef_df.tail(2))

    features      coef
931    vegan  0.706615
57    animal  0.189788
    features      coef
150    carbs -0.308900
479     keto -0.746959


A positive coefficient points towards the vegan class while a negative coefficient points to the keto class. 

Besides the obvious words of vegan and keto, we can interpret the coefficient as follows:
- animal -> As the word count for animal increases by 1, our classifier is e^0.434 = 1.54 times more likely to classify it as a vegan post
- carbs ->  As the word count for carbs increases by 1, our classifier is e^0.766 = 2.15 times more likely to classify it as a vegan post

#### Generate Predictions

In [21]:
preds = gs_1.predict(X_test)

### Classification Metrics

#### Confusion Matrix

In [22]:
tn, fp, fn, tp = confusion_matrix(y_test,preds).ravel()
print('True Negative:' ,tn)
print('False Positive (Type I):' ,fp)
print('False Negative (Type II):' ,fn)
print('True Positive:' ,tp)

True Negative: 125
False Positive (Type I): 11
False Negative (Type II): 2
True Positive: 191


In [23]:
#Precision

prec = tp/(tp+fp)

print(f'Precision: {prec}')

Precision: 0.9455445544554455


#### Precision
Precision measures primarily on type I errors or false positives. This means that the model classifies a post as a vegan post when it is actually a keto post.

Our precision score is 0.94 and this means that our model rarely mislabels a keto post as a vegan post. 

The shortcoming of this is that it cannot measure false negatives, classifying keto posts as vegan posts

In [24]:
#Recall/Sensitivity

sens = tp/(tp+fn)

print(f'Sensitivity: {sens}')

Sensitivity: 0.9896373056994818


#### Recall/Sensitivity
Recall focuses on type II errors. A type II error means that the model classifies a post as a keto post when it is actually a vegan post.

Our precision score is 1.0 and this means that our model never mislabels a vegan post as a keto post. 

Vice versa, the shortcoming of this metric is that it cannot measure false positives. 

In [25]:
#let's take a look at ROC AUC Score yet another metric
roc_auc_score(y_test, preds)

0.9543774763791527

#### ROC AUC Score
Our high ROC AUC score of 0.96 (close to 1) confirms that we have a good separation between our vegan and keto classes. 

In [26]:
print(classification_report(y_test,preds))

             precision    recall  f1-score   support

          0       0.98      0.92      0.95       136
          1       0.95      0.99      0.97       193

avg / total       0.96      0.96      0.96       329



#### F1 Score
The F1 score shows the balance between precision and recall. It is calculated by 2(precision*recall)/(precision+recall).
Our high F1 score of 0.96 shows that we have a good balance of both of the above. 

Ultimately, we measure our success on the F1 score which generally works well with unbalanced classes, which we have at 58/42. 
Also, we dont particularly value identifying either false positive or false negatives more, Accuracy and F1 scores should be sufficient to measure our success. 

### Recommendations

Our reccomendations are: 
- further develop the classifier to continuously improve our core product of generating data analytics  

With this proof of concept, we can invest further resources to develop the model to eventually classify all available data, to contribute to our core product of nutrition data related services.  

- vegan insights: focus on educating the target market

With vegan subredditors, we noticed that the users were highly discerning. Afterall, vegans have made a life changing commitment to the lifestlye to avoid meat and animal products. They are concerned with making the world a better place. In order to rally these users, frequently post about the science behind veganism, share insights on how they can contribute to reduce animal cruelty and keep them updated on the change that veganism is bringing.

- keto insights: focus on building a strong community

With keto subredditors, a common goal/motivation is weight loss. Build a community with these people, share progress, recipes and tips. Build a platform when people can freely share their thoughts and concerns. 

### Conclusion
There were 2 keys questions that we set out to answer in our problem statement:
1. Can we build a classifier, with at least 90% accuracy, to sort text data from Keto and Vegan subreddits?
2. Can we draw insights from these text data for targeted marketing?

After data cleaning, feature engineering and EDA, we ran our text data through a couple of vectorizer-model combinations. We tuned the hyperparameters to generate the best scores. From there, we scored them on the training data, using the accuracy scores. We chose the model with the highest accuracy score and scored our optimal model with the test data. Finally, our classifier achieved a test score of 0.964, which means that it correctly classified text data 96.4% of the time.

We also drew insights from the EDA and profiled our target market. We realised differences in motivations and preferences between the 2 groups. Our recommendations are tailored to each group with a focus on educating for the vegans and building a strong community for the keto crowd. 

With these, we will be able to provide better insights to our clients to help with their marketing campaigns. 

#### Next steps

To further improve our model, I believe that we could have included texts from the comment section which were not availabble via the API. We could perhaps explore using PRAW to scrap comments. 

We could also run sentiment analysis on the data to see which posts and comments were favoured by different users of the subreddits. 